In [1]:
import cv2
import os
import numpy as np
from PIL import Image, ImageEnhance
image_path1 = r"H:\LUMS\Dr-Murtaza-Taj\Camera-Calibration\brandenburg_gate\dense\images\00581890_3574867299.jpg"
image_path2 = r"H:\LUMS\Dr-Murtaza-Taj\Camera-Calibration\brandenburg_gate\dense\images\01738801_5114523193.jpg"
target_size = (150, 150)
target_mean = 127.5
target_std = 80
processed_images = []
for image_path in [image_path1, image_path2]:
    if not os.path.exists(image_path):
        print(f"Error: Image not found at: {image_path}")
        continue
    img = cv2.imread(image_path)
    if img is None:
       print(f"Error: Failed to load image at: {image_path}")
       continue
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    current_mean = np.mean(img_gray)
    current_std = np.std(img_gray)  
    if current_std!=0:
        scale_factor = target_std / current_std
        brightness_offset = target_mean - current_mean * scale_factor
        img_adjusted = cv2.convertScaleAbs(img, alpha=scale_factor, beta=brightness_offset)
    else:
        img_adjusted = img
    pil_img = Image.fromarray(cv2.cvtColor(img_adjusted, cv2.COLOR_BGR2RGB))
    resized_img = pil_img.resize(target_size, Image.Resampling.LANCZOS)
    processed_images.append(resized_img)
if len(processed_images) == 2:
    processed_images[0].show(title="Processed Image 1")
    processed_images[1].show(title="Processed Image 2")
    

In [33]:
import os
import numpy as np

file_path_cameras = r"brandenburg_gate\neuralsfm\cameras.txt"
k_matrices = []
with open(file_path_cameras, 'r') as file:
    lines = file.readlines()
    relevant_lines = lines[3:1366]
    for line in relevant_lines:
        parts = line.strip().split()
        camera_id = int(parts[0])
        model = parts[1].strip()
        width = int(parts[2])
        height = int(parts[3])
        params = list(map(float, parts[4:]))
        if len(params) < 4:
            continue
        
        f_x, f_y, c_x, c_y = params[:4]
        K = np.array([
            [f_x, 0, c_x],
            [0, f_y, c_y],
            [0, 0, 1]
        ])
        k_matrices.append((camera_id, K))
intrinsics  = []        
intrinsics.append(k_matrices[3][1])
intrinsics.append(k_matrices[17][1])

def quaternion_to_rotation_matrix(qw, qx, qy, qz):
    R = np.array([
    [1 - 2*(qy**2 + qz**2), 2*(qx*qy - qz*qw), 2*(qx*qz + qy*qw)],
    [2*(qx*qy + qz*qw), 1 - 2*(qx**2 + qz**2), 2*(qy*qz - qx*qw)],
    [2*(qx*qz - qy*qw), 2*(qy*qz + qx*qw), 1 - 2*(qx**2 + qy**2)]
    ])
    return R

file_path_images = r"brandenburg_gate\neuralsfm\images.txt"
camera_poses = []
with open(file_path_images, 'r') as file:
    lines = file.readlines()
    for i in range(4, len(lines), 2): 
        line1 = lines[i].strip()
        line2 = lines[i+1].strip() if i + 1 < len(lines) else ""
        
        if line1.startswith('#') or not line1 or line2.startswith('#') or not line2:
            continue
        parts1 = line1.split()
        parts2 = line2.split()
        image_id = int(parts1[0]) 
        qw, qx, qy, qz = map(float, parts1[1:5]) 
        tx, ty, tz = map(float, parts1[5:8])  
        R = quaternion_to_rotation_matrix(qw, qx, qy, qz)  
        t = np.array([tx, ty, tz]).reshape(3,1) 
        camera_poses.append((image_id, R, t)) 

combined_poses = {}
indices = [3, 17]
for i, (image_id, R, t) in enumerate(camera_poses):
    if i in indices:
        RT = np.concatenate((R, t), axis=1)
        combined_poses[image_id] = RT

# --- Constraint Verification Functions ---
def verify_rotation_constraints(R, tolerance=1e-6):
    r1 = R[0, :]
    r2 = R[1, :]
    r3 = R[2, :]
    if (abs(np.dot(r1, r2)) > tolerance or
        abs(np.dot(r1, r3)) > tolerance or
        abs(np.dot(r2, r3)) > tolerance):
        return False
    identity = np.eye(3)
    if not np.all(np.abs(np.dot(R, R.T) - identity) < tolerance):
        return False
    if abs(np.linalg.det(R) - 1) > tolerance:
       return False
    return True

def verify_projection_constraints(P, tolerance=1e-6):
  vx = P[:3, 0] / P[2,0]
  vy = P[:3, 1] / P[2,1]
  vz = P[:3, 2] / P[2,2]
  wc = P[:3, 3] / P[2,3]
  if (abs(vx[0]) < tolerance or abs(vx[1]) < tolerance or
      abs(vy[0]) < tolerance or abs(vy[1]) < tolerance or
          abs(vz[0]) < tolerance or abs(vz[1]) < tolerance):
      return False
  return True

for i, (image_id, RT) in enumerate(combined_poses.items()):
    print(f"Camera pose for image (ID = {image_id}):")
    print(f"  RT matrix:\n{RT}")
    if i < len(intrinsics):
        K = intrinsics[i]
        print(f"  Intrinsic matrix:\n{K}")
        R = RT[:,:3]
        t = RT[:,3:]
        P = np.dot(K, np.concatenate((R, t), axis=1))
        rotation_valid = verify_rotation_constraints(R)
        projection_valid = verify_projection_constraints(P)
        print(f"  Rotation constraints valid: {rotation_valid}")
        print(f"  Projection constraints valid: {projection_valid}")

Camera pose for image (ID = 1363):
  RT matrix:
[[ 0.84066541 -0.09035049 -0.53396484  3.4599676 ]
 [ 0.10270195  0.99469017 -0.00661611 -0.20256423]
 [ 0.53172735 -0.0492773   0.8454808  -0.57750759]]
  Intrinsic matrix:
[[1.24885779e+03 0.00000000e+00 5.16500000e+02]
 [0.00000000e+00 1.24885779e+03 3.42500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
  Rotation constraints valid: True
  Projection constraints valid: True
Camera pose for image (ID = 697):
  RT matrix:
[[ 8.95122025e-01 -5.96068065e-02 -4.41818503e-01  2.84904576e+00]
 [ 6.77420807e-02  9.97699366e-01  2.64306598e-03  7.96587957e-02]
 [ 4.40644495e-01 -3.22955713e-02  8.97100566e-01  5.58065177e-01]]
  Intrinsic matrix:
[[834.0489502   0.        517.       ]
 [  0.        834.0489502 385.5      ]
 [  0.          0.          1.       ]]
  Rotation constraints valid: True
  Projection constraints valid: True


In [68]:
import base64
import google.generativeai as genai
from IPython.display import Markdown

model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp")

image_path_1 = r"H:\LUMS\Dr-Murtaza-Taj\Camera-Calibration\brandenburg_gate\dense\images\00581890_3574867299.jpg"
image_path_2 = r"H:\LUMS\Dr-Murtaza-Taj\Camera-Calibration\brandenburg_gate\dense\images\01738801_5114523193.jpg"

def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

sample_file_1 = {
    "mime_type": "image/jpeg",
    "data": encode_image(image_path_1)
}

sample_file_2 = {
    "mime_type": "image/jpeg",
    "data": encode_image(image_path_2)
}

prompt = "The intrinsic matrix for Image 1 is given as:\n\n" \
         "[[1.24885779e+03 0.00000000e+00 5.16500000e+02]\n" \
         " [0.00000000e+00 1.24885779e+03 3.42500000e+02]\n" \
         " [0.00000000e+00 0.00000000e+00 1.00000000e+00]]\n\n" \
         "Given this, the image 2 is similar to image 1. Try to predict the intrinsic matrix for Image 2 in the same format. Do not repeat the exact first. :\n\n" \
         "Intrinsic Matrix (Image 2):\n[[f_x, 0, c_x], [0, f_y, c_y], [0, 0, 1]]\n" \
         "Do not include additional explanations."

response = model.generate_content([sample_file_1, sample_file_2, prompt])

Markdown(response.text)


[[1.24886000e+03 0.00000000e+00 5.17000000e+02]
 [0.00000000e+00 1.24886000e+03 3.43000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [67]:
import base64
import google.generativeai as genai
from IPython.display import Markdown

model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp")

image_path_1 = r"brandenburg_gate\dense\images\00581890_3574867299.jpg"
image_path_2 = r"brandenburg_gate\dense\images\01738801_5114523193.jpg"

def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

sample_file_1 = {
    "mime_type": "image/jpeg",
    "data": encode_image(image_path_1)
}

sample_file_2 = {
    "mime_type": "image/jpeg",
    "data": encode_image(image_path_2)
}

prompt = "The intrinsic matrix for Image 1 is given as:\n\n" \
         "[[1.24885779e+03 0.00000000e+00 5.16500000e+02]\n" \
         " [0.00000000e+00 1.24885779e+03 3.42500000e+02]\n" \
         " [0.00000000e+00 0.00000000e+00 1.00000000e+00]]\n\n" \
         "Given this, the image 2 is similar to image 1. Try to predict the intrinsic matrix for Image 2 in the same format. fx fy values for image 2 are in range 700 to 1000. Do not average the range come with some apporoximation using the image :\n\n" \
         "Intrinsic Matrix (Image 2):\n[[f_x, 0, c_x], [0, f_y, c_y], [0, 0, 1]]\n" \
         "Do not include additional explanations."

response = model.generate_content([sample_file_1, sample_file_2, prompt])

Markdown(response.text)


[[900.0, 0.0, 550.0], [0.0, 900.0, 370.0], [0.0, 0.0, 1.0]]

<h1>With Pre-Processing</h1>

In [73]:
import base64
import cv2
import google.generativeai as genai
from IPython.display import Markdown

model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp")

image_path_1 = r"brandenburg_gate\dense\images\00581890_3574867299.jpg"
image_path_2 = r"brandenburg_gate\dense\images\01738801_5114523193.jpg"

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray_image, (150, 150))
    _, encoded_image = cv2.imencode('.jpg', resized_image)
    return base64.b64encode(encoded_image).decode("utf-8")

sample_file_1 = {
    "mime_type": "image/jpeg",
    "data": preprocess_image(image_path_1)
}

sample_file_2 = {
    "mime_type": "image/jpeg",
    "data": preprocess_image(image_path_2)
}

prompt = "The intrinsic matrix for Image 1 is given as:\n\n" \
         "[[1.24885779e+03 0.00000000e+00 5.16500000e+02]\n" \
         " [0.00000000e+00 1.24885779e+03 3.42500000e+02]\n" \
         " [0.00000000e+00 0.00000000e+00 1.00000000e+00]]\n\n" \
         "Given this, the image 2 is similar to image 1. Try to predict the intrinsic matrix for Image 2 in the same format. fx fy values for image 2 are in range 700 to 1000. Do not average the range, come with some approximation using the image:\n\n" \
         "Intrinsic Matrix (Image 2):\n[[f_x, 0, c_x], [0, f_y, c_y], [0, 0, 1]]\n" \
         "Do not include additional explanations."

response = model.generate_content([sample_file_1, sample_file_2, prompt])

Markdown(response.text)


[[1.24885779e+03, 0.00000000e+00, 5.16500000e+02], [0.00000000e+00, 1.24885779e+03, 3.42500000e+02], [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]


In [72]:
import base64
import cv2
import google.generativeai as genai
from IPython.display import Markdown

model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp")

image_path_1 = r"brandenburg_gate\dense\images\00581890_3574867299.jpg"
image_path_2 = r"brandenburg_gate\dense\images\01738801_5114523193.jpg"

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray_image, (150, 150))
    _, encoded_image = cv2.imencode('.jpg', resized_image)
    return base64.b64encode(encoded_image).decode("utf-8")

sample_file_1 = {
    "mime_type": "image/jpeg",
    "data": preprocess_image(image_path_1)
}

sample_file_2 = {
    "mime_type": "image/jpeg",
    "data": preprocess_image(image_path_2)
}

prompt = """The intrinsic matrix for Image 1 is given as:

[[1.24885779e+03 0.00000000e+00 5.16500000e+02]
 [0.00000000e+00 1.24885779e+03 3.42500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Image 2 is captured under similar conditions to Image 1. The scene is similar, and both images use the same camera type with a focal length in the range of 700 to 1000. The camera positions may vary slightly. Based on these factors, predict the intrinsic matrix for Image 2.

Intrinsic Matrix (Image 2):
[[f_x, 0, c_x], [0, f_y, c_y], [0, 0, 1]]
Do not include additional explanations."""

response = model.generate_content([sample_file_1, sample_file_2, prompt])

Markdown(response.text)


[[900, 0, 516], [0, 900, 342], [0, 0, 1]]


In [82]:
import base64
import cv2
import google.generativeai as genai
from IPython.display import Markdown

model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp")

image_path_1 = r"brandenburg_gate\dense\images\00581890_3574867299.jpg"
image_path_2 = r"brandenburg_gate\dense\images\01738801_5114523193.jpg"

def preprocess_image_with_features_and_matches(image_path_1, image_path_2):
    image_1 = cv2.imread(image_path_1)
    image_2 = cv2.imread(image_path_2)
    gray_image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    gray_image_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create()
    keypoints_1, descriptors_1 = orb.detectAndCompute(gray_image_1, None)
    keypoints_2, descriptors_2 = orb.detectAndCompute(gray_image_2, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors_1, descriptors_2)
    matches = sorted(matches, key = lambda x:x.distance)

    img_with_matches = cv2.drawMatches(image_1, keypoints_1, image_2, keypoints_2, matches[:30], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    _, encoded_image = cv2.imencode('.jpg', img_with_matches)
    return base64.b64encode(encoded_image).decode("utf-8")

sample_file_1 = {
    "mime_type": "image/jpeg",
    "data": preprocess_image_with_features_and_matches(image_path_1, image_path_2)
}

sample_file_2 = {
    "mime_type": "image/jpeg",
    "data": preprocess_image_with_features_and_matches(image_path_1, image_path_2)
}

prompt = """The intrinsic matrix for Image 1 is given as:

[[1.24885779e+03 0.00000000e+00 5.16500000e+02]
 [0.00000000e+00 1.24885779e+03 3.42500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Image 2 is captured under similar conditions to Image 1, though there may be slight differences in the camera position or lens orientation. The focal length of Image 2 is likely in the range of 700 to 900, but it may differ slightly from Image 1. The scene and lighting are similar, but there might be small variations in the image, especially considering the slight difference in camera position. Based on these factors, predict the intrinsic matrix for Image 2 that best reflects the small variations between the two images.

Intrinsic Matrix (Image 2):
[[f_x, 0, c_x], [0, f_y, c_y], [0, 0, 1]]
the values for f_x and f_y differ slightly from Image 1 based on the image features and matches. Do not include additional explanations."""

response = model.generate_content([sample_file_1, sample_file_2, prompt])

Markdown(response.text)


```
[[800.0, 0.0, 516.5], [0.0, 800.0, 342.5], [0.0, 0.0, 1.0]]
```